<h1>Hello <i>kiara</i>.</h1>

This tutorial introduces *kiara*, a data orchestration software.
It will walk you through installation of the software in Jupyter Notebooks, and some basic but essential functions that can be built on in further notebooks.

This tutorial requires you to know **python** and **SQL**.


<h2>Installation</h2>

First, we need to check if *kiara* and its plugins are already installed, and install them if not. There are currently seven plugins:

- `kiara_plugin.core-types`
- `kiara_plugin.onboarding`
- `kiara_plugin.tabular`
- `kiara_plugin.network_analysis`
- `kiara_plugin.language_processing`
- `kiara_plugin.html`
- `kiara_plugin.streamlit`

All of these will be installed automatically alongside *kiara*, using the code below.

<span style="color:blue">\* Note: would be great if this could link out to further documentation, with a separate section for each plugin, describing overall what is in each, then with specific details of the functions in each</span>

In [1]:
try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()


⠧ Installing kiara plugins...
⠧ Installing kiara plugins...

╭─ kiara plugin(s) installed ─────────────────────────────────────────────────────────────────────────────────────╮
│ Installed packages:                                                                                             │
│                                                                                                                 │
│  • kiara-plugin.onboarding                                                                                      │
│  • kiara-plugin.html                                                                                            │
│  • kiara-plugin.streamlit                                                                                       │
│                                                                                                                 │
│ Depending on the state of your current environment, you might see error messages below, in which case you'll    │
│ have to restart the jupyter kernel manually.                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<h2>Running <i>kiara</i></h2>

In order to use *kiara*, we need to create a `KiaraAPI` instance. An API allows us to control and interact with *kiara* and its functions. In *kiara* this also allows us to get more information about what can be done (and what is happening) to our data as we go.

<span style="color:blue">\* Note: I'm calling the API 'kiara' at the moment because I think it simplifies it for people who don't know what an API is/how it functions and might get confused by the variable name. This keeps it grounded in 'kiara' (whilst indicating it's an API for those who want to know/already know more) - the first instance of kiara isn't used again in this notebook, but we can always reverse this if it makes things more complicated for follow-on notebooks!</span>

In [2]:
from kiara import KiaraAPI

kiara = KiaraAPI.instance()

Now we have an API in place, we can get more information about what we can do in *kiara*. Let's start by asking *kiara* to list all the operations that are included with the plugins we just installed.

The documentation for each of these functions can be found [here](https://dharpa.org/kiara/latest/reference/kiara/interfaces/python_api/__init__/#kiara.interfaces.python_api.KiaraAPI)

In [3]:
kiara.list_operation_ids()

['create.database.from.file',
 'create.database.from.file_bundle',
 'create.database.from.table',
 'create.network_data.from.files',
 'create.network_data.from.tables',
 'create.stopwords_list',
 'create.table.from.file',
 'create.table.from.file_bundle',
 'date.check_range',
 'date.extract_from_string',
 'download.file',
 'download.file_bundle',
 'export.file.as.file',
 'export.network_data.as.csv_files',
 'export.network_data.as.graphml_file',
 'export.network_data.as.sql_dump',
 'export.network_data.as.sqlite_db',
 'export.table.as.csv_file',
 'extract.date_array.from.table',
 'file_bundle.pick.file',
 'file_bundle.pick.sub_folder',
 'filter.table',
 'generate.LDA.for.tokens_array',
 'import.database.from.local_file_path',
 'import.file',
 'import.file_bundle',
 'import.local.file',
 'import.local.file_bundle',
 'import.network_data.from.local_file_paths',
 'import.table.from.local_file_path',
 'import.table.from.local_folder_path',
 'jupyter.example',
 'kiara_plugin.my_kiara_module

<h3>Downloading Files</h3>

Great, now we know the different kind of operations we can use with *kiara*. Let's start by introducing some files to our notebook, using the `download.file` function.<br/>
First we want to find out what this operation does, and just as importantly, what inputs it needs to work.

In [4]:
kiara.retrieve_operation_info('download.file')

Documentation                                                                                                                              
                     Download a single file from a remote location.                                                                         
                                                                                                                                            
                     The result of this operation is a single value of type 'file' (basically an array of raw bytes), which can then be     
                     used in other modules to create more meaningful data structures.                                                       
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         onboarding                                                                                                
                     Labels       package: kiara_plugin.onboarding                                                                          
                     References   source_repo: ]8;id=63891;https://github.com/DHARPA-Project/kiara_plugin.onboarding\https://github.com/DHARPA-Project/kiara_plugin.onboarding]8;;\                                    
                                  documentation: ]8;id=744835;https://DHARPA-Project.github.io/kiara_plugin.onboarding/\https://DHARPA-Project.github.io/kiara_plugin.onboarding/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Download a single file from a remote location.                                                         
                                                                                                                                            
                                     The result of this operation is a single value of type 'file' (basically an array of raw bytes),       
                                     which can then be used in other modules to                                                             
                                     create more meaningful data structures.                                                                
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       url          string   The url of the file to download.               yes        -- no default --     
                                       file_name    string   The file name to use for the downloaded        no         -- no default --     
                                   

So from this, we know that `download.file` will download a single file from a remote location for us to use in *kiara*. <br/>
We need to give the function a **url** and, if we want, a **file name**. These are the <span style="color:green">**inputs**</span>. <br/>
In return, we will get the **file** and **metadata** about the file as our <span style="color:red">**outputs**</span>.

Let's give this a go using some *kiara* sample data.

First we define our <span style="color:green">inputs</span>, then use `kiara.run_job` with our chosen operation, `download.file`, and save this as our <span style="color:red">outputs</span>.

In [5]:
inputs = {
        "url": "https://raw.githubusercontent.com/DHARPA-Project/kiara.examples/main/examples/data/journals/JournalNodes1902.csv",
        "file_name": "JournalNodes1902.csv"
}

outputs = kiara.run_job('download.file', inputs=inputs)

Let's print out our <span style="color:red">outputs</span> and see what that looks like.

In [6]:
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field               value                                                                                                              │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   download_metadata                                                                                                                      │
│                         dict data     {                                                                                                  │
│                                         "response_headers": [                                                                            │
│                                           {                                                                                              │
│                                             "connection": "keep-alive",                                                                  │
│                                             "content-length": "7436",                                                                    │
│                                             "cache-control": "max-age=300",                                                              │
│                                             "content-security-policy": "default-src 'none'; style-src 'unsafe-inline'; sandbox",         │
│                                             "content-type": "text/plain; charset=utf-8",                                                 │
│                                             "etag": "W/\"641ae85d69e5836d27ea8906aba0a33b48b0f3ed0ed4c40d21a07fccebdd238d\"",            │
│                                             "strict-transport-security": "max-age=31536000",                                             │
│                                             "x-content-type-options": "nosniff",                                                         │
│                                             "x-frame-options": "deny",                                                                   │
│                                             "x-xss-protection": "1; mode=block",                                                         │
│                                             "x-github-request-id": "BCA0:8395:120207:18015C:63D13F8F",                                   │
│                                             "content-encoding": "gzip",                                                                  │
│                                             "accept-ranges": "bytes",                                                                    │
│                                             "date": "Wed, 25 Jan 2023 14:41:19 GMT",                                                     │
│                                             "via": "1.1 varnish",                                                                        │
│                                             "x-served-by": "cache-mad22020-MAD",                                                         │
│                                             "x-cache": "MISS",                                                                           │
│                                             "x-cache-hits": "0",                                                                         │
│                                             "x-timer": "S1674657679.217538,VS0,VE249",                                                   │
│                                             "vary": "Authorization,Accept-Encoding,Origin",                                              │
│                                             "acces

Great! We've successfully downloaded the file, and we can see there's lots of information here.

At the moment, we're most interested in the **file** output. This contains the actual *contents* of the file that we have just downloaded.

Let's separate this out and store it in a separate variable for us to use.

In [7]:
downloaded_file = outputs['file']

<h3>New Formats: Creating and Converting</h3>

What next? We could transform the downloaded file contents into a different format. <br/>
Let's use the operation list earlier, and look for something that allows us to create something out of our new file.

In [8]:
kiara.list_operation_ids('create')

['create.database.from.file',
 'create.database.from.file_bundle',
 'create.database.from.table',
 'create.network_data.from.files',
 'create.network_data.from.tables',
 'create.stopwords_list',
 'create.table.from.file',
 'create.table.from.file_bundle']

Our file was orginally in a CSV format, so let's make a table using `create.table.from.file`. 

Just like when we used `download.file`, we can double check what this does, and what <span style="color:green">inputs</span> and <span style="color:red">outputs</span> this involves.

This time, we're also going to use a variable to store the operation in - this is especially handy if the operation has a long name, or if you want to use the same operation more than once without retyping it.

In [9]:
op_id = 'create.table.from.file'

kiara.retrieve_operation_info(op_id)

Documentation                                                                                                                              
                     Create a table from a file, trying to auto-determine the format of said file.                                          
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=745704;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=476438;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a table from a file, trying to auto-determine the format of said file.                          
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type   description                                      Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file         file   The source value (of type 'file').               yes        -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name   type    description                                                                     
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table        table   The result value (of type 'table').                                             
                                                                                                                                            
                  

Great, we have all the information we need now.

Let's go again.

First we define our <span style="color:green">inputs</span>, the downloaded file we saved earlier.

Then use `kiara.run_job` with our chosen operation, this time stored as `op_id`.

Once this is saved as our <span style="color:red">outputs</span>, we can print it out.

In [10]:
inputs = {
    "file": downloaded_file
}

outputs = kiara.run_job(op_id, inputs=inputs)

outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   table                                                                                                                                  │
│             Id    Label            JournalType      City        CountryNetworkT   PresentDayCoun   Latitude    Longitude    Language     │
│            ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│             75    Psychiatrische   specialized: p   Amsterdam   Netherlands       Netherlands      52.366667   4.9          Dutch        │
│             36    The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             208   The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             295   Die Krankenpfl   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             296   Die deutsche K   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             300   Therapeutische   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             1     Allgemeine Zei   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             7     Archiv für Psy   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             10    Berliner klini   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             13    Charité Annale   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             21    Monatsschrift    specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             29    Virchows Archi   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             31    Zeitschrift fü   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             42    Vierteljahrssc   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             47    Centralblatt f   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             50    Russische medi   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             277   L'arte medica    general medici   Turin       Italy             Italy            45.079167   7.676111     Italian      │
│             288   Allgemeine öst   specialized: a   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             18    Jahrbücher für   specialized: p   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             30    Wiener klinisc   general medic

This has done exactly what we wanted, and shown the contents from the downloaded file as a table. But we are also interested in some general (mostly internal) information and metadata, this time for the new table we have just created, rather than the original file itself.

Let's have a look.

In [11]:
outputs_table = outputs['table']

outputs_table

 value_id            a3b2a75c-eb1d-473e-8fbd-d1a9a5affaff                                                                                  
  kiara_id            76b2fda2-bbef-451c-aebf-3bf7a0e3e707                                                                                  
                                                                                                                                            
                      ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  
  data_type_info                                                                                                                            
                        data_type_name     table                                                                                            
                        data_type_config   {}                                                                                               
                        characteristics    {                                                                                                
                                             "is_scalar": false,                                                                            
                                             "is_json_serializable": false                                                                  
                                           }                                                                                                
                        data_type_class                                                                                                     
                                             python_class_name    TableType                                                                 
                                             python_module_name   kiara_plugin.tabular.data_types.table                                     
                                             full_name            kiara_plugin.tabular.data_types.table.TableType                           
                                                                                                                                            
                                                                                                                                            
  destiny_backlinks   {}                                                                                                                    
  enviroments         None                                                                                                                  
  property_links      {                                                                                                                     
                        "metadata.python_class": "7e45026e-0369-42ce-ac78-635d4a4d9eb8",                                                    
                        "metadata.table": "aed134cf-0aa7-4c70-a20e-c4f72d2b4ee4"                                                            
                      }                                                                                                                     
  value_hash          zdpuAn89Et1ENzfoASJRYcWEceyfRiPg664mN4nnHLFnjRLyg                                                                     
  value_schema                                                                                                                              
                        type          table                                                                                                 
                        type_config   {}                                                                                                    
                        default       __not_set__                                                                                           
                        optional      False          

<h3>Querying our Data</h3>

So now we have downloaded our file and converted it into a table, we want to actually explore it.

To do this, we can query the table using **SQL** and some functions already included in *kiara*.

Let's take another look at that operation list, this time looking for functions that let us 'query'.


In [12]:
kiara.list_operation_ids('query')

['query.database', 'query.table']

Well, we already know our file has been converted into a table, so let's have a look at `query.table`.

In [13]:
kiara.retrieve_operation_info('query.table')

Documentation                                                                                                                              
                     Execute a sql query against an (Arrow) table.                                                                          
                                                                                                                                            
                     The default relation name for the sql query is 'data', but can be modified by the 'relation_name' config               
                     option/input.                                                                                                          
                                                                                                                                            
                     If the 'query' module config option is not set, users can provide their own query, otherwise the pre-set one will be   
                     used.                                                                                                                  
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=631017;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=294140;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Execute a sql query against an (Arrow) table.                                                          
                                                                                                                                            
                                     The default relation name for the sql query is 'data', but can be modified by the 'relation_name'      
                                     config option/input.                                                                                   
                                                                                                                                            
                                     If the 'query' module config option is not set, users can provide their own query, otherwise the       
                                     pre-set                                                                                                
                                     one will be used.                                                                                      
                                        

So from this information, we only need to provide the **table** itself, and our **query**.

Let's work out how many of these journals were published in Berlin.

In [14]:
inputs = {
    "table": outputs_table
    "query": "SELECT * from data where City like 'Berlin'"
}

outputs = kiara.run_job('query.table', inputs=inputs)

outputs

SyntaxError: invalid syntax (586479838.py, line 2)

The function has returned the table with just the results we were looking for from the SQL query. 

Let's narrow this further, and find all the journals that are just about general medicine and published in Berlin.

We can re-use the `query.table` function and the table we've just made, stored in `outputs['query_result']`

In [ ]:
inputs = {
    "table" : outputs['query_result'],
    "query" : "SELECT * from data where JournalType like 'general medicine'"
}

outputs = kiara.run_job('query.table', inputs=inputs)

outputs

<h3>Recording and Tracing our Data</h3>

We've quite a few changes to this table, so let's double check the information about this new table we've created with our queries.

In [ ]:
query_output = outputs['query_result']

query_output

Looks good!

We might have changed things around, but we can still get lots of information about all our data.

More importantly, *kiara* is able to trace all of these changes, tracking the inputs and outputs and giving them all different identifiers, so you know exactly what has happened to your data. <br/>Check it out!

In [ ]:
query_output.lineage

Even though we are only actually asking for the **data lineage** using the *last* SQL query and the table it made, *kiara* shows us everything that has happened since we first downloaded the file. This helps us keep an eye on the research process *and* the changes we are making to the data at the same time!

<span style="color:blue"> \* Note: this can be updated/made more detailed esp. once we have Mariella's visualisations</span>

<h3>What next...?</h3>

That's great, you've completed the first notebook and successfully installed *kiara*, downloaded files, tested out some functions, and are able to see what this does to your data. 

Now you can check out the other plugin packages to explore how this helps you manage and trace your data while using digital analysis tools!